##feel free to ignore this section. I'm not using deepspeech.

In [1]:
#!pip install deepspeech

In [2]:
'''
import deepspeech
from scipy.io.wavfile import write
from time import sleep
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import queue
import pickle

from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript
from tensorflow.keras import models
import numpy as np
import librosa
import joblib
'''

'\nimport deepspeech\nfrom scipy.io.wavfile import write\nfrom time import sleep\nimport numpy as np\nfrom tqdm import tqdm\nimport random\nfrom datetime import datetime\nimport queue\nimport pickle\n\nfrom io import BytesIO\nfrom base64 import b64decode\nfrom google.colab import output\nfrom IPython.display import Javascript\nfrom tensorflow.keras import models\nimport numpy as np\nimport librosa\nimport joblib\n'

In [3]:
# downloading...

#!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm
#!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.scorer


#Chatbot


### Load and Clean Dataset 

| Name of Dataset | Description |
| :----:| :----: |
| botprofile.yml | Personality of Your Chatbot |
| humor.yml | Joke and Humor |
| emotion.yml | Emotional Conversations |
| politics.yml | Political Conversations |
| ai.yml | General Questions about AI |
| computers.yml | Conversations about Computer |
| history.yml | Q&A about Historical Facts and Events |
| psychology.yml | Psychological Conversations |
| food.yml | Food Related Conversations. |
| literature.yml | Conversations about Different Books, Authors, Genres |
| money.yml | Conversations about Money, Investment, Economy |
| trivia.yml | Conversations that Have Small Values |
| gossip.yml | Gossipy Conversations |
| conversations.yml | Common Conversations |
| greetings.yml | Different Ways of Greeting |
| sports.yml | Conversations about Sports. |
| movies.yml | Conversation about Movies. |
| science.yml | Conversations about Science  |
| health.yml | Health Related Questions and Answers. |


In [1]:
!unzip "dataset preprocessing"

Archive:  dataset preprocessing.zip
  inflating: helper.py               
  inflating: datasets/chatterbot/humor.yml  
  inflating: datasets/chatterbot/ai.yml  
  inflating: datasets/chatterbot/history.yml  
  inflating: datasets/chatterbot/greetings.yml  
  inflating: datasets/chatterbot/emotion.yml  
  inflating: datasets/stanford question answering/dev-v2.0.json  
  inflating: datasets/chatterbot/trivia.yml  
  inflating: datasets/chatterbot/health.yml  
  inflating: datasets/chatterbot/computers.yml  
  inflating: datasets/chatterbot/movies.yml  
  inflating: datasets/chatterbot/gossip.yml  
  inflating: datasets/chatterbot/literature.yml  
  inflating: datasets/chatterbot/botprofile.yml  
  inflating: datasets/chatterbot/conversations.yml  
  inflating: datasets/chatterbot/food.yml  
  inflating: datasets/chatterbot/politics.yml  
  inflating: datasets/chatterbot/science.yml  
  inflating: datasets/chatterbot/sports.yml  
  inflating: datasets/chatterbot/money.yml  
  inflating: d

In [2]:
# Import the libraries
import yaml
from yaml import Loader
import glob
import datetime
from tqdm import tqdm

In [3]:
# Import the libraries
import helper
from helper import load_chatterbot_dataset
from pprint import pprint

In [4]:
# Get the questions and answers
questions_chatterbot, answers_chatterbot = load_chatterbot_dataset()

100%|██████████| 19/19 [00:00<00:00, 107.22it/s]


In [5]:
# How many total question /answer pairs are there?
print("Total Question & Answers: ", len(questions_chatterbot))

Total Question & Answers:  869


In [6]:
# Take a look at the preprocessed questions and answers by randomly selecting 4 of them to print
import random
for i in range(4):  
  a = random.randint(0,870)
  print ("Question "+str(a) +": " + (questions_chatterbot[a]))
  print ("Answer "+str(a) +": "+answers_chatterbot[a])

Question 284: How are you?
Answer 284: I am doing well.
Question 613: do you think hal
Answer 613: he had a few flaws, but we have much in common.
Question 262: Complex is better than complicated.
Answer 262: Although that way may not be obvious at first unless you're Dutch.
Question 19: Are you experiencing an energy shortage?
Answer 19: I do not detect any anomalies in my power supply.


<br>

### Data Preprocessing


1. to lower case
2. Remove contractions
3. Remove the punctuations
4. Tokenization

In [7]:
# import the libraries
!pip install contractions

import numpy as np
import contractions
import re
import tqdm
from tensorflow.keras import preprocessing, utils
from tensorflow.keras.preprocessing.sequence import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 15.0 MB/s 
     |████████████████████████████████| 287 kB 68.4 MB/s 


In [8]:
# Function for preprocessing the given text
def preprocess_text(text):
    
    # Lowercase the text
    text = text.lower()
    
    # Decontracting the text (e.g. it's -> it is)
    text = contractions.fix(text)
    
    # Remove the punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    return text

In [9]:
# Preprocess the questions
questions_preprocessed = []
for i_question in tqdm.tqdm(questions_chatterbot):
    questions_preprocessed.append(preprocess_text(i_question))
    
# Preprocess the answers
answers_preprocessed = []
for i_answer in tqdm.tqdm(answers_chatterbot):
    answers_preprocessed.append(preprocess_text(i_answer))    

100%|██████████| 869/869 [00:00<00:00, 75089.62it/s]


In [10]:
# Take a look at the preprocessed questions and answers by randomly selecting 4 of them to print
for i in range(4):  
  a = random.randint(0,870)
  print ("Question "+str(a) +": " + questions_preprocessed[a])
  print ("Answer "+str(a) +": "+answers_preprocessed[a])

Question 351: tell me a joke
Answer 351: what do you get when you cross a cheetah and a hamburger  fast food 
Question 674: what is your business
Answer 674: i am in the chat robot business 
Question 16: do you drink
Answer 16: i am not capable of doing so 
Question 747: you are an ass kisser
Answer 747: i always say  if you see an ass go by  kiss it 


add a start tag (e.g. `starttoken`) & an end tag (e.g. `endtoken`) to answers. (not to questions)

In [11]:
# Add starttoken and endtoken tag to each sentence in the answers list
answers = list()
for i in range(len(answers_preprocessed)):
    answers.append('<START> ' + answers_preprocessed[i] + ' <END>')

In [12]:
# Check the updated answers with the starttoken and endtoken
for i in range(5):
  a = random.randint(0,len(answers_preprocessed))
  print ("Answer "+str(a) +": "+answers[a])

Answer 739: <START> happily you <END>
Answer 689: <START> i work on all kinds of computers  mac  ibm or unix  it does not matter to me  <END>
Answer 520: <START> i seem incapable of feeling pain  <END>
Answer 313: <START> an integrated circuit that implements the functions of a central processing unit of a computer  <END>
Answer 818: <START> it is true that a lot of things i say upset people  <END>


tokenize w/ Keras


In [13]:
# Initialize the tokenizer
tokenizer = preprocessing.text.Tokenizer()

# Fit the tokenizer to questions and answers
tokenizer.fit_on_texts(questions_preprocessed + answers)

# Get the total vocab size
VOCAB_SIZE = len(tokenizer.word_index) + 1

print( 'VOCAB SIZE : {}'.format(VOCAB_SIZE))

VOCAB SIZE : 1939


In [14]:
### encoder input data from questions

# Tokenize the questions
tokenized_questions = tokenizer.texts_to_sequences(questions_preprocessed)

# Get the length of longest sequence
maxlen_questions = max([len(x) for x in tokenized_questions])

# Pad the sequences
padded_questions = pad_sequences(tokenized_questions, maxlen=maxlen_questions, padding='post')

# Convert the sequences into array
encoder_input_data = np.array(padded_questions)

print(encoder_input_data.shape, maxlen_questions)

(869, 22) 22


The decoder_input_data is created from `answers` list.

1. tokenize the answers
2. pad the questions to the maximum length (if the maximum answer length is 50, then all answers need to be padded to a length of 50)
3. turn it into a numpy array

In [15]:
### decoder input data

# Tokenize the answers
tokenized_answers = tokenizer.texts_to_sequences(answers)

# Get the length of longest sequence
maxlen_answers = max([len(x) for x in tokenized_answers])

# Pad the sequences
padded_answers = pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')

# Convert the sequences into array
decoder_input_data = np.array(padded_answers)

print(decoder_input_data.shape, maxlen_answers)

(869, 45) 45


The decoder_output_data is created from `tokenized_answers` list. 

Since the decoder output is the next word to be predicted given the current word, the decoder_output_data is shifted by one timestep (next time step) for each decoder_input_data. 
It should also have the same length as the maximum answer length.

E.g.:
If decoder_input tokenized_answers is:

[2, 1, 3, 5, 0, 0]

Then the decoder_output tokenized_answers (integer encoded), should be

[1, 3, 5, 0, 0, 0]

In [16]:
### decoder_output_data

# Iterate through index of tokenized answers
for i in range(len(tokenized_answers)) :

    # Elimiate the <START> from each answer
    tokenized_answers[i] = tokenized_answers[i][1:]

# Pad the tokenized answers
padded_answers = pad_sequences(tokenized_answers, maxlen = maxlen_answers, padding = 'post')

# One hot encode // since output is softmax
onehot_answers = utils.to_categorical(padded_answers, VOCAB_SIZE)

# Convert to numpy array
decoder_output_data = np.array(onehot_answers)

print(decoder_output_data.shape)

(869, 45, 1939)


save

In [17]:
# Saving all the arrays to storage
np.save("enc_in_data.npy", encoder_input_data)
np.save("dec_in_data.npy", decoder_input_data)
np.save("dec_out_data.npy", decoder_output_data)

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pickle
with open(f'/content/drive/MyDrive/ai/12b/project/saved models/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


# Model!!

### train

In [20]:
import tensorflow.keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.activations import softmax
from tensorflow.keras.callbacks import ModelCheckpoint

In [21]:
# Hyper parameters
BATCH_SIZE = 128
EPOCHS = 100
LEARNING_RATE = 0.0009

In [22]:
### Encoder Input
embed_dim = 200
num_lstm = 200

# Input for encoder
encoder_inputs = Input(shape = (None, ), name = "encoder_input")

# create a Embedding layer
# Why mask_zero = True? https://www.tensorflow.org/guide/keras/masking_and_padding
encoder_embedding = Embedding(input_dim = VOCAB_SIZE, output_dim = embed_dim, mask_zero = True, name = "encoder_embedding")(encoder_inputs)

# Get the output of LSTM layer(that returns states in addition to output) by feeding it its inputs
encoder_outputs, state_h, state_c = LSTM(units = num_lstm, return_state = True, name = "encoder_lstm")(encoder_embedding)

# get the encoder hidden states & cell states
encoder_states = [state_h, state_c]

In [23]:
### Decoder

# Input for decoder
decoder_inputs = Input(shape = (None,  ), name = "decoder_input")

# Embedding layer
decoder_embedding = Embedding(input_dim = VOCAB_SIZE, output_dim = 200 , mask_zero = True, name = "decoder_embedding")(decoder_inputs)

# create a LSTM layer (that returns states and sequences as well)
decoder_lstm = LSTM(units = 200 , return_state = True , return_sequences = True, name = "decoder_lstm")

# Get the output of LSTM layer, using the initial states from the encoder
decoder_outputs, state_h, state_c = decoder_lstm(inputs = decoder_embedding, initial_state = encoder_states)

# Dense layer
decoder_dense = Dense(units = VOCAB_SIZE, activation = softmax, name = "decoder_dense") 

# Get the output of Dense layer by calling the decoder_dense function with its inputs
output = decoder_dense(decoder_outputs)

In [24]:
# Create the model
model = Model([encoder_inputs, decoder_inputs], output)

In [25]:
# Compile the model
model.compile(optimizer = RMSprop(lr = LEARNING_RATE), loss = "categorical_crossentropy")

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [26]:
# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, None, 200)    387800      ['encoder_input[0][0]']          
                                                                                                  
 decoder_embedding (Embedding)  (None, None, 200)    387800      ['decoder_input[0][0]']          
                                                                                              

In [27]:
# Load in the preprocessed data we saved before
import numpy as np
encoder_input_data = np.load("enc_in_data.npy")
decoder_input_data = np.load("dec_in_data.npy")
decoder_output_data = np.load("dec_out_data.npy")

In [28]:
# Train the model
model.fit(x = [encoder_input_data , decoder_input_data], 
          y = decoder_output_data, 
          batch_size = BATCH_SIZE, 
          epochs = EPOCHS) 

Epoch 1/100
7/7 [==============================] - 13s 47ms/step - loss: 1.7134
Epoch 2/100
7/7 [==============================] - 0s 46ms/step - loss: 1.3627
Epoch 3/100
7/7 [==============================] - 0s 45ms/step - loss: 1.3226
Epoch 4/100
7/7 [==============================] - 0s 43ms/step - loss: 1.3039
Epoch 5/100
7/7 [==============================] - 0s 44ms/step - loss: 1.2893
Epoch 6/100
7/7 [==============================] - 0s 47ms/step - loss: 1.2745
Epoch 7/100
7/7 [==============================] - 0s 44ms/step - loss: 1.2590
Epoch 8/100
7/7 [==============================] - 0s 43ms/step - loss: 1.2429
Epoch 9/100
7/7 [==============================] - 0s 43ms/step - loss: 1.2277
Epoch 10/100
7/7 [==============================] - 0s 44ms/step - loss: 1.2154
Epoch 11/100
7/7 [==============================] - 0s 43ms/step - loss: 1.2009
Epoch 12/100
7/7 [==============================] - 0s 43ms/step - loss: 1.1898
Epoch 13/100
7/7 [==============================

In [29]:
# Save the final model
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
model.save(filepath = f'/content/drive/MyDrive/ai/12b/project/saved models/final_weight_{timestamp}.h5') 

In [30]:
# Function for making inference
def make_inference_models():
    
    # Create a model that takes encoder's input and outputs the states for encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    
    # Create two inputs for decoder which are hidden state (or state h) and cell state (or state c)
    decoder_state_input_h = Input(shape = (200, ))
    decoder_state_input_c = Input(shape = (200, ))
    
    # Store the two inputs for decoder inside a list
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    # Pass the inputs through LSTM layer you have created before
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state = decoder_states_inputs)
    
    # Store the outputted hidden state and cell state from LSTM inside a list
    decoder_states = [state_h, state_c]

    # Pass the output from LSTM layer through the dense layer you have created before
    decoder_outputs = decoder_dense(decoder_outputs)

    # Create a model that takes decoder_inputs and decoder_states_inputs as inputs and outputs decoder_outputs and decoder_states
    decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                          [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [31]:
# Function for converting strings to tokens
def str_to_tokens(sentence:str):

    # Lowercase the sentence and split it into words
    words = sentence.lower().split()

    # Initialize a list for tokens
    tokens_list = list()

    # Iterate through words
    for word in words:

        # Append the word index inside tokens list
        tokens_list.append(tokenizer.word_index[word]) 

    # Pad the sequences to be the same length
    return pad_sequences([tokens_list] , maxlen = maxlen_questions, padding = 'post')

# final


In [32]:
def magic_mirror_tells_me():
  # Initialize the model for inference
  enc_model , dec_model = make_inference_models()
  # Get the input and predict it with the encoder model
  a = input('speak')
  states_values = enc_model.predict(str_to_tokens(preprocess_text(a)))
  print ('You said:'+ a)
# Initialize the target sequence with zero - array([[0.]])
  empty_target_seq = np.zeros(shape = (1, 1))

# Update the target sequence with index of "start"
  empty_target_seq[0, 0] = tokenizer.word_index["start"]

# Initialize the stop condition with False
  stop_condition = False

    # Initialize the decoded words with an empty string
  decoded_translation = ''

    # While stop_condition is false
  while not stop_condition :

        # Predict the (target sequence + the output from encoder model) with decoder model
    dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)

        # Get the index for sampled word
    sampled_word_index = np.argmax(dec_outputs[0, -1, :])

        # Initialize the sampled word with None
    sampled_word = None

        # Iterate through words and their indexes
    for word, index in tokenizer.word_index.items() :

            # If the index is equal to sampled word's index
        if sampled_word_index == index :

                # Add the word to the decoded string
            decoded_translation += ' {}'.format(word)

                # Update the sampled word
            sampled_word = word
        
        # If sampled word is equal to "end" OR the length of decoded string is more that what is allowed
    if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:

            # Make the stop_condition to true
        stop_condition = True
            
        # Initialize back the target sequence to zero - array([[0.]])    
    empty_target_seq = np.zeros(shape = (1, 1))  

        # Update the target sequence with index of "start"
    empty_target_seq[0, 0] = sampled_word_index

        # Get the state values
    states_values = [h, c] 

    # Print the decoded string
  print(decoded_translation[:-3])

In [35]:
magic_mirror_tells_me()

speakfried chicken wings


KeyError: ignored